## Step-by-step Guide to Installing TensorFlow-GPU 2.17 on Mac M3 Max Silicon Chip

### Author: Dr. Saad Laouadi

#### Date: July 27, 2024

Setup Steps
===========

1. **Install Homebrew**: If you haven’t installed Homebrew, you can do so by running the following command in your terminal:
```sh
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
```

2. **Install Miniforge**: Miniforge is a minimal conda installer that is compatible with M3 Macs.
   ```sh
   brew install --cask miniforge
   ```
   
3. **Create a Conda Environment**: Create a conda environment for TensorFlow using the `mamba` package manager. This helps manage dependencies and isolate your TensorFlow installation from other Python projects.
    ```sh
    mamba create --name tf-gpu-env
    ```
4. **Activate the newly created environment**:
    ```sh
    mamba activate tf-gpu-env
    # upgrade pip 
    pip install --upgrade pip
    ```

5. **Install the latest version of TensorFlow**:
    ```sh
    python3 -m pip install tensorflow~=2.17 tf_keras~=2.17
    ```
6. **Install Tensorflow Metal**: Tensorflow metal is required to access the GPU:
    ```sh
    python3 -m pip install tensorflow-metal
    ```
    
7. **Check the installation**:
    ```sh
    pip list | grep -E 'keras|tensorflow'
    ```
   - **Here is the output on my machine:**
    ```text
      keras                        3.4.1
      tensorflow                   2.17.0
      tensorflow-datasets          4.8.3+nightly
      tensorflow-estimator         2.8.0
      tensorflow-io-gcs-filesystem 0.37.1
      tensorflow-metadata          1.13.1
      tensorflow-metal             1.1.0
      tf_keras                     2.17.0
    ```
    
Setup JupyterLab and Register the Kernel
========================================

1. **Install JupyterLab web-based application**:
    ```sh
    python3 -m pip install jupyterlab
    ```
2. **Install JupyterLab Desktop application**: Alternatively, you can install the JupyterLab desktop application (if you don't already have that):
    ```sh
    brew install --cask jupyterlab
    ```
3. **Register the kernel to be available for JupyterLab**:
    ```sh
    python3 -m pip install ipython
    ipython kernel install --user --name=tf-gpu-env --display-name=TF-GPU-2.17
    ```
    
Test the Environment
====================

1. **Create a working directory for a new project**:
    ```sh
    mkdir test-project && cd test-project
    ```
    
2. **Open JupyterLab**:

   - Web-based:
        ```sh
        jupyterlab .
        ```

   - Desktop JupyterLab:
        ```sh
        open -a jupyterlab .
        ```

In [1]:
#=================================================================================#
#             Testing Tensorflow on Mac M3 Max 
#=================================================================================#
import sys
import tensorflow as tf

# Check if the current Python executable matches the expected path
expected_executable = '/opt/homebrew/Caskroom/mambaforge/base/envs/tfenv/bin/python3'
assert sys.executable == expected_executable, f"Expected Python executable: {expected_executable}, but got: {sys.executable}"

# Check if the TensorFlow version starts with "2.17"
assert tf.__version__.startswith('2.17'), f"Expected TensorFlow version to start with '2.17', but got: {tf.__version__}"

# Print system and TensorFlow version information
print("="*72)
print(f'System: {sys.executable}')
print(f'TensorFlow: {tf.__version__}')
print("="*72)

# Check the available physical Devices
for device in ['CPU', 'GPU']:
    print(tf.config.list_physical_devices(device))

print('*'*72)

System: /opt/homebrew/Caskroom/mambaforge/base/envs/tfenv/bin/python3
TensorFlow: 2.17.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
************************************************************************


## GPU-Testing 

In [3]:
%%time

# Ensure we see the GPU in device list.
print('Visible Devices: ', tf.config.get_visible_devices())

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=1, batch_size=128)

Visible Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
391/391 ━━━━━━━━━━━━━━━━━━━━ 164s 329ms/step - accuracy: 0.0593 - loss: 4.7821
CPU times: user 3min 4s, sys: 44.6 s, total: 3min 49s
Wall time: 2min 45s


## CPU-Test

In [11]:
# Restart the kernel so we can test on CPU
%%capture
!jupyter notebook stop 57134    # The port this notebook is running 

In [1]:
%%time

import tensorflow as tf
# Removes GPU from list, i.e. []
tf.config.set_visible_devices([], 'GPU')
print('Visible Devices: ', tf.config.get_visible_devices())

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=1, batch_size=128)

Visible Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
391/391 ━━━━━━━━━━━━━━━━━━━━ 496s 1s/step - accuracy: 0.0573 - loss: 4.8759
CPU times: user 20min 43s, sys: 4min 45s, total: 25min 28s
Wall time: 8min 18s


### References

1. [What's new in TensorFlow 2.16](https://blog.tensorflow.org/2024/03/whats-new-in-tensorflow-216.html)
2. [Get started with tensorflow-metal](https://developer.apple.com/metal/tensorflow-plugin/)
3. [Homebrew](https://brew.sh/)
4. [Tensorflow](https://www.tensorflow.org/)
5. [Miniforge](https://github.com/conda-forge/miniforge)